# Plotting DNA Order Data using pandas and Bokeh

Table of contents
----
1. Load and clean the data
2. Calculate summary statistics
3. Create Graphs

Benson notes: created on 3-14-2018

Note for when Filemaker ODBC/JDBC is working -
    SQL equivalent to tsv file used here:
```
SELECT 
	DNAOrdItem.ABRC_stock_number, 
	order_id_tair, date_item_shipped, 
	"s.number_in_set",
	"org.country" 
	FROM DNAOrdItem
LEFT JOIN "Lab_File by org_id" on "org.id" = Org_id_from_order
LEFT JOIN Stock_DNA on DNAOrdItem.ABRC_stock_number = Stock_DNA.ABRC_stock_number
WHERE date_item_shipped > DATE '2013-01-01'
```

### 1. Load and clean the data

##### Import the necessary libraries:
* pandas
* bokeh

In [100]:
import pandas as pd
from bokeh.layouts import Column, Row
from bokeh.models.widgets import CheckboxButtonGroup, Panel, Tabs, DataTable, TableColumn, NumberFormatter, DateFormatter
from bokeh.models import HoverTool, DataRange1d, DatetimeTickFormatter, CustomJS
from bokeh.plotting import figure, show, ColumnDataSource, Row, output_file
from bokeh.palettes import Category10
from bokeh.io import output_notebook

In [3]:
# Analysis Parameters

# name of the data file to use
dna_export_filename = 'data/data_dna_3-15-18.tab'
# start date (ignore orders shipped before this date)
start_date = "01-01-2013"
# end date (ignore orders shipped after this date)
end_date = "2-28-2018"
# threshold for sets
sets_threshold = 500

# Plotting options

# plot size in pixels
plot_height = 480
plot_width = 640
# color pallette (stick with category 10 to match seed lab data)
current_pallette = Category10
# name to use for the graphs file
graphs_title = "DNA Orders 3-15-2018 500 tubes"

Now load the data from a tab-separated file exported from Filemaker DNAOrdItem

In [4]:
# load data file
raw_data = pd.read_csv(dna_export_filename, sep='\t', header=None)
# assign names to each column
raw_data.columns = ['stock_num', 'order_num', 'date', 'delete_this', 'stock_type', 'num_tubes', 'country']
# fill in missing data by propagating foward from the last good data point
raw_data.fillna(method='ffill', inplace=True)
# remove any duplicate entries in the file
raw_data.drop_duplicates(inplace=True)
# we need to explicitly set out "date" column as a date instead of just text
raw_data.date = pd.to_datetime(raw_data.date)
# we need to convert the "num_tubes" column to an integer (right now it is a decimal like 1.0)
raw_data['num_tubes'] = raw_data['num_tubes'].astype('int')

**Here is what our data should look like at this stage**

*Notice the column named delete_this?* 

That column is actually date order shipped, we don't need it so we'll delete it in a moment

 |stock_num | order_num|	date|	delete_this|	stock_type|	num_tubes|	country
-|----------|----------|--------|--------------|--------------|----------|--------------
0|	CD264610|	79725|	2011-03-09|	3/9/2011|	RNAi|	1|	USA
1|	CD267010|	79725|	2011-03-09|	3/9/2011|	RNAi|	1	|USA
2|	CD255587|	79978|	2011-03-09|	4/6/2011|	RNAi|	1	|USA
3|	CD259213|	79978|	2011-04-06|	4/6/2011|	RNAi|	1|	USA
4|	CD260785|	79978|	2011-04-06|	4/6/2011|	RNAi|	1	|USA

In [5]:
# preview the data
raw_data.head()

,stock_num,order_num,date,delete_this,stock_type,num_tubes,country
0,CD258385,90123,2013-11-25,2/14/2012,RNAi,1,Korea
1,CD271532,91812,2013-02-06,2/6/2013,RNAi,1,Argentina
2,CD255709,91877,2013-02-06,2/6/2013,RNAi,1,Denmark
3,CD254446,91952,2013-02-06,5/2/2012,RNAi,1,USA
4,CD259696,92001,2013-02-06,2/6/2013,RNAi,1,People's Republic of China


-------
Select only items shipped within the time range we chose, then discard the rest

In [6]:
# remove items shipped before our start date
raw_data = raw_data.loc[raw_data['date'] >= pd.to_datetime(start_date)]
# remove items shipped after our end date
raw_data = raw_data.loc[raw_data['date'] <= pd.to_datetime(end_date)]

In [7]:
raw_data.head()

,stock_num,order_num,date,delete_this,stock_type,num_tubes,country
0,CD258385,90123,2013-11-25,2/14/2012,RNAi,1,Korea
1,CD271532,91812,2013-02-06,2/6/2013,RNAi,1,Argentina
2,CD255709,91877,2013-02-06,2/6/2013,RNAi,1,Denmark
3,CD254446,91952,2013-02-06,5/2/2012,RNAi,1,USA
4,CD259696,92001,2013-02-06,2/6/2013,RNAi,1,People's Republic of China


##### Removing edge cases and weird stuff 
**Next, there are a couple of anomolies in the data that need to come out**

For example, these express shipping charges aren't useful to us


 |stock_num | order_num|	date|	delete_this|	stock_type|	num_tubes|	country
-|----------|----------|--------|--------------|--------------|----------|--------------
22276|	XPDUS|	97620|	2013-01-08|	1/8/2013|	none|	1|	USA|
22877|	XPDFOR|	98064|	2013-01-28|	1/28/2013|	none|	1|	Turkey|
23250|	XPDFOR|	98344|	2013-02-11|	2/11/2013|	none|	1|	Chile|
23498|	XPDFOR|	98525|	2013-02-25|	2/25/2013|	none|	1|	India|
23825|	XPDFORDI|	98785|	2013-03-11|	3/11/2013|	none|	1|	Korea|

In [8]:
# clean data by removing services and education kits and various other abnormalities

# removing stocks without a stock type e.g. XPDUS
raw_data = raw_data.loc[raw_data['stock_type'] != 'none']
# remove the education kit data and keep it in a separate dataset
edukits = raw_data.loc[raw_data['stock_type'] == 'education_kit']
raw_data = raw_data.loc[raw_data['stock_type'] != 'education_kit']
# remove services such as freezer rentals
raw_data = raw_data[~raw_data['stock_num'].str.contains('SERVICE')]
# drop this order that was placed twice for billing reasons (this was done intentionally)
raw_data = raw_data.loc[raw_data['order_num'] != 109485]

Our data is now ready to perform some statistics!

In [9]:
# preview the data
raw_data.head()

,stock_num,order_num,date,delete_this,stock_type,num_tubes,country
0,CD258385,90123,2013-11-25,2/14/2012,RNAi,1,Korea
1,CD271532,91812,2013-02-06,2/6/2013,RNAi,1,Argentina
2,CD255709,91877,2013-02-06,2/6/2013,RNAi,1,Denmark
3,CD254446,91952,2013-02-06,5/2/2012,RNAi,1,USA
4,CD259696,92001,2013-02-06,2/6/2013,RNAi,1,People's Republic of China


### 2. Perform summary statistics

First, we'll split our `raw_data` up into `df_small` and `df_sets`
**"df" stands for DataFrame**

All stocks that are a single tubes or a group of tubes smaller than 500 will be part of `df_small` and stocks that are sets of 500 or more tubes will be part of `df_sets`

In [10]:
# less than 500
df_small = raw_data[raw_data.num_tubes < sets_threshold]
# more than 500
df_sets = raw_data[raw_data.num_tubes >= sets_threshold]

Now that they are separated, lets work on `df_small`. First we want to aggregate the data by order number and get the total number of items per order.

In [11]:
# group by order_num, then sum up number of tubes per order
temp_df=df_small.groupby('order_num').sum().reset_index()
# lets peek at the result
temp_df.head()

,order_num,num_tubes
0,90123,1
1,91812,1
2,91877,1
3,91952,1
4,92001,4


Our data should now look similar to this:

         
order_num | num_tubes
------|-----
90123|	1
91812|	1
91877|	1
91952|	1
92001|	4

Now we can start building our new data frame with the summary information. We'll name it `df_summary`

In [12]:
# separately save the meta_data for each order
# then we can add it back later
metadata = df_small[['order_num', 'date', 'country']].drop_duplicates()
metadata.head()

,order_num,date,country
0,90123,2013-11-25,Korea
1,91812,2013-02-06,Argentina
2,91877,2013-02-06,Denmark
3,91952,2013-02-06,USA
4,92001,2013-02-06,People's Republic of China


In [13]:
# add back metadata for each order
temp_df = temp_df.join(metadata.set_index('order_num'), how='left', on='order_num')
# make an additional column with just the year
temp_df['year'] = temp_df.date.dt.year

In [14]:
# preview the data
temp_df.head()

,order_num,num_tubes,date,country,year
0,90123,1,2013-11-25,Korea,2013
1,91812,1,2013-02-06,Argentina,2013
2,91877,1,2013-02-06,Denmark,2013
3,91952,1,2013-02-06,USA,2013
4,92001,4,2013-02-06,People's Republic of China,2013


Now we will convert the data to monthly totals and calculate the cumulative totals

In [15]:
# create an empty dataframe with a row for each month from our start date to our finish date
df_summary = pd.DataFrame(index=pd.date_range(start=start_date, end=end_date, freq='M'))
# calculate the number of tubes shipped per month by grouping by month and
# summing up the number of tubes for all the orders in each month
# then store that calculation in our new dataframe
df_summary['tubes'] = temp_df.set_index('date').resample('M').sum().num_tubes
# now we need to fill in zeros for months where nothing was ordered
df_summary.fillna(inplace=True, value=0)
# next calculate the cumulative sum of both orders and items, then merge it into our dataframe
df_summary=df_summary.join(df_summary.groupby(df_summary.index.year).cumsum(), rsuffix='_cumul')
# lets take a look now
df_summary.head()

,tubes,tubes_cumul
2013-01-31,1062,1062
2013-02-28,805,1867
2013-03-31,586,2453
2013-04-30,955,3408
2013-05-31,795,4203


In [16]:
# now break the date out into multiple columns which we will use for labels on our graphs
df_summary['year'] = df_summary.index.year # e.g. 2013, 2014
df_summary['month'] = pd.to_datetime(df_summary.index.month, format='%m') # e.g. 1900-01-01, 1900-02-01
df_summary['str_date'] = df_summary.index.strftime('%b-%Y') # e.g. Jan-2013, Feb-2013
df_summary['month_name'] = df_summary.index.strftime('%b') # e.g. Jan, Feb
# preview again
df_summary.head()

,tubes,tubes_cumul,year,month,str_date,month_name
2013-01-31,1062,1062,2013,1900-01-01,Jan-2013,Jan
2013-02-28,805,1867,2013,1900-02-01,Feb-2013,Feb
2013-03-31,586,2453,2013,1900-03-01,Mar-2013,Mar
2013-04-30,955,3408,2013,1900-04-01,Apr-2013,Apr
2013-05-31,795,4203,2013,1900-05-01,May-2013,May


Now that our number of items excluding large sets (>500 tubes) dataset is ready, we can do the exact process for `df_sets`

**This time I have condensed everything into one cell**


*Benson note: this will need adjusted in case someone has ordered more than one set in a single order*

In [17]:
# group by order_num, then sum up number of tubes per order
temp_df=df_sets.groupby('order_num').sum().reset_index()
# lets peek at the result
temp_df.head()

# separately save the meta_data for each order
# then we can add it back later
metadata = df_sets[['order_num', 'date', 'country']].drop_duplicates()
metadata.head()

# add back metadata for each order
df_sets = temp_df.join(metadata.set_index('order_num'), how='left', on='order_num')
# make an additional column with just the year
df_sets['year'] = df_sets.date.dt.year

# create an empty dataframe with a row for each month from our start date to our finish date
df_sets_summary = pd.DataFrame(index=pd.date_range(start=start_date, end=end_date, freq='M'))
# calculate the number of stocks per month by grouping by month and counting the number of rows in each month
# then store that calculation in our new dataframe
df_sets_summary['stocks'] = df_sets.set_index('date').resample('M').count().order_num
# ditto for number of tubes, except this time we take a sum the number of tubes for all the orders in a month
df_sets_summary['tubes'] = df_sets.set_index('date').resample('M').sum().num_tubes
# now we need to fill in zeros for months where nothing was ordered
df_sets_summary.fillna(inplace=True, value=0)
# next calculate the cumulative sum of both orders and items, then merge it into our dataframe
df_sets_summary = df_sets_summary.join(df_sets_summary.groupby(df_sets_summary.index.year).cumsum(), rsuffix='_cumul')


# now break the date out into multiple columns which we will use for labels on our graphs
df_sets_summary['year'] = df_sets_summary.index.year # e.g. 2013, 2014
df_sets_summary['month'] = pd.to_datetime(df_sets_summary.index.month, format='%m') # e.g. 1900-01-01, 1900-02-01
df_sets_summary['str_date'] = df_sets_summary.index.strftime('%b-%Y') # e.g. Jan-2013, Feb-2013
df_sets_summary['month_name'] = df_sets_summary.index.strftime('%b') # e.g. Jan, Feb

df_sets_summary.tail()

,stocks,tubes,stocks_cumul,tubes_cumul,year,month,str_date,month_name
2017-10-31,1.0,2034.0,2.0,3990.0,2017,1900-10-01,Oct-2017,Oct
2017-11-30,0.0,0.0,2.0,3990.0,2017,1900-11-01,Nov-2017,Nov
2017-12-31,1.0,2034.0,3.0,6024.0,2017,1900-12-01,Dec-2017,Dec
2018-01-31,2.0,3990.0,2.0,3990.0,2018,1900-01-01,Jan-2018,Jan
2018-02-28,0.0,0.0,2.0,3990.0,2018,1900-02-01,Feb-2018,Feb


##### Next step we need to get the cumulative number of orders
This is basically the same as the two datasets above, except we need to go back to our original `raw_data` dataset, so that we are including all the orders

**Again, this is condensed into a single cell**

In [18]:
# group by order_num, then sum up number of tubes per order
temp_df=raw_data.groupby('order_num').sum().reset_index()
# lets peek at the result
temp_df.head()

# separately save the meta_data for each order
# then we can add it back later
metadata = raw_data[['order_num', 'date', 'country']].drop_duplicates()
metadata.head()

# add back metadata for each order
df_orders = temp_df.join(metadata.set_index('order_num'), how='left', on='order_num')
# make an additional column with just the year
df_orders['year'] = df_orders.date.dt.year

# create an empty dataframe with a row for each month from our start date to our finish date
df_orders_summary = pd.DataFrame(index=pd.date_range(start=start_date, end=end_date, freq='M'))
# calculate the number of orders per month by grouping by month and counting the number of rows in each month
# then store that calculation in our new dataframe
df_orders_summary['orders'] = df_orders.set_index('date').resample('M').count().order_num
# ditto for number of tubes, except this time we take a sum the number of tubes for all the orders in a month
df_orders_summary['tubes'] = df_orders.set_index('date').resample('M').sum().num_tubes
# now we need to fill in zeros for months where nothing was ordered
df_orders_summary.fillna(inplace=True, value=0)
# next calculate the cumulative sum of both orders and items, then merge it into our dataframe
df_orders_summary = df_orders_summary.join(df_orders_summary.groupby(df_orders_summary.index.year).cumsum(), rsuffix='_cumul')


# now break the date out into multiple columns which we will use for labels on our graphs
df_orders_summary['year'] = df_orders_summary.index.year # e.g. 2013, 2014
df_orders_summary['month'] = pd.to_datetime(df_orders_summary.index.month, format='%m') # e.g. 1900-01-01, 1900-02-01
df_orders_summary['str_date'] = df_orders_summary.index.strftime('%b-%Y') # e.g. Jan-2013, Feb-2013
df_orders_summary['month_name'] = df_orders_summary.index.strftime('%b') # e.g. Jan, Feb

df_orders_summary.tail()

,orders,tubes,orders_cumul,tubes_cumul,year,month,str_date,month_name
2017-10-31,98,2528,1059,9260,2017,1900-10-01,Oct-2017,Oct
2017-11-30,97,432,1156,9692,2017,1900-11-01,Nov-2017,Nov
2017-12-31,68,2437,1224,12129,2017,1900-12-01,Dec-2017,Dec
2018-01-31,94,4459,94,4459,2018,1900-01-01,Jan-2018,Jan
2018-02-28,128,542,222,5001,2018,1900-02-01,Feb-2018,Feb


In [19]:
## Section 3: Create graphs

We now have three datasets that will be used for 3 different graphs. Since these graphs are all really similar we'll define a common graphing method that we can reuse for each dataset.

# we need this to see the graphs in this notebook
output_notebook()

In [20]:
output_notebook()

Loading BokehJS ...

In [21]:
def setup_plot(source, title, value_to_plot, y_axis_title):
    tools_to_show = 'hover,save,reset,wheel_zoom'
    plot = figure(x_axis_type="datetime", height=plot_height, width=plot_width, tools=tools_to_show, toolbar_sticky=False)
    plot.title.text = title
    plot.xaxis[0].formatter = DatetimeTickFormatter(months=['%b'])
    start_year = min(source.index).year
    end_year = max(source.index).year
    years = [y for y in range(start_year, end_year + 1)]
    year_lines = dict()
    # create a new subset of the data for each year and use that to draw a line for that year
    for y in years:
        dy = ColumnDataSource(data=source[str(y)])
        year_lines[str(y)] = plot.line(x="month", y=value_to_plot, source=dy, line_width=8,
                                       line_color=current_pallette[len(years)][years.index(y)], legend=str(y), muted_alpha=0.2, muted_color=current_pallette[len(years)][years.index(y)])

    # set formatting options for the plot
    plot.legend.location = "top_left"
    # fixed attributes
    plot.xaxis.axis_label = None
    # label each month with a tick, otherwise months are skipped
    plot.xaxis[0].ticker.desired_num_ticks = 12
    plot.yaxis.axis_label = y_axis_title
    plot.xaxis.axis_label = """Time (months)"""
    plot.axis.axis_label_text_font_style = "bold"
    plot.x_range = DataRange1d(range_padding=0.00)
    plot.xgrid.grid_line_alpha = 0.5
    plot.ygrid.grid_line_alpha = 0.5
    plot.xgrid.grid_line_color = 'black'
    plot.ygrid.grid_line_color = 'black'
    hover = plot.select(dict(type=HoverTool))
    hover.tooltips = [("Month", "@str_date"), ("# of " + value_to_plot, "@" + value_to_plot)]
    hover.mode = 'mouse'
    hover.attachment = 'horizontal'
    hover.point_policy = 'snap_to_data'
    hover.line_policy = 'nearest'
    plot.xaxis.major_label_orientation = 45
    plot.xaxis.major_label_text_font_size = '20pt'
    plot.yaxis.major_label_text_font_size = '20pt'
    plot.axis.axis_label_text_font_style = 'bold'
    
    plot.xaxis.axis_label_text_font_style = 'bold'
    if plot_height >= 720:
        plot.legend.glyph_height = 50
        plot.legend.glyph_width = 50
        plot.legend.label_text_font_size = '14pt'
        plot.yaxis.axis_label_text_font_size = '16pt'
        plot.xaxis.axis_label_text_font_size = '16pt'
    else:
        plot.legend.glyph_height = 20
        plot.legend.glyph_width = 20
        plot.legend.label_text_font_size = '10pt'
        plot.yaxis.axis_label_text_font_size = '14pt'
    plot.legend.click_policy = 'mute'
    return plot

In [22]:
# create a dictionary to store the layouts by name
layouts = dict()

In [23]:
# to create a plot we just just need to call setup_plot and tell it what the data source, title, and value to plot are

plot = setup_plot(source=df_summary, title="DNA Plasmids - no sets", value_to_plot='tubes_cumul', y_axis_title="""DNA Plasmids (excld. sets > 500)""")
# then call our checkbox function

# finally create a layout row with the plot and checkboxes

layouts["DNA Tubes - no sets"] = plot


In [24]:
# to see the plot, uncomment the line below by removing the '#' from the beginning
show(layouts["DNA Tubes - no sets"])

In [25]:
# now for sets
plot = setup_plot(source=df_sets_summary, title="DNA Plasmids - sets only", value_to_plot='tubes_cumul', y_axis_title="""DNA Plasmids (sets > 500 only)""")

layouts["DNA Tubes - sets only"] = plot

In [26]:
# to see the plot, uncomment the line below by removing the '#' from the beginning
show(layouts["DNA Tubes - sets only"])

In [27]:
# and for orders
plot = setup_plot(source=df_orders_summary, title="DNA Orders", value_to_plot='orders_cumul', y_axis_title="DNA Orders")
layouts["DNA Orders"] = plot

In [28]:
# to see the plot, uncomment the line below by removing the '#' from the beginning
show(layouts["DNA Orders"])

## Last Step - Add Summary Tables and export to excel

Let's add a summary table below each plot and also export the data to excel.

The summary needs to be in a more human friendly format.

Instead of this format: 

 |tubes|	tubes_cumul|	year|	month|	str_date|	month_name
--|--------------------|--------|--------|----------|----------
2013-01-31|	1062	|1062|	2013	|1900-01-01|	Jan-2013	|Jan
2013-02-28|	805	|1867	|2013	|1900-02-01	|Feb-2013	|Feb
2013-03-31|	586	|2453	|2013	|1900-03-01	|Mar-2013	|Mar
2013-04-30|	955	|3408	|2013	|1900-04-01	|Apr-2013	|Apr
2013-05-31|	795	|4203	|2013	|1900-05-01	|May-2013	|May

We'll "pivot" the data so that each column is a year and a row is a month.

**Like this: **

month|	2013|	2014|	2015|	2016|	2017|	2018
----|-------|-------|-------|-------|-------|--------
1900-01-01|	1062.0	|654.0|	408.0|	408.0|	682.0|	469.0
1900-02-01	|805.0	|775.0|	394.0|	686.0|	491.0|	542.0
1900-03-01	|586.0	|592.0|	670.0|	541.0|	747.0|	0.0
1900-04-01	|955.0	|790.0|	1186.0|	679.0|	515.0|	0.0
1900-05-01	|795.0	|675.0|	702.0|	518.0|	433.0|	0.0

*(Note that first column dates are all 1900-XX-01, I'm going to convert this to Jan, Feb, etc)*

*Right now the months don't stay in order, so maybe 1-Jan, 2-Feb, etc will work*

In [31]:
# create matrix style summaries 
dfp1 = df_summary.pivot(index='month', columns='year', values='tubes').fillna(value=0)
dfp2 = df_sets_summary.pivot(index='month', columns='year', values='tubes').fillna(value=0)
dfp3 = df_orders_summary.pivot(index='month', columns='year', values='orders').fillna(value=0)


In [82]:
# create a "panel" which contains all three summary dataframes 
# this will let us save them all together as an individual sheets in a single excel workbook
panel_summary = pd.Panel({'DNA Tubes - no sets':dfp1, 'DNA Tubes - sets only':dfp2, 'DNA Orders':dfp3})

In [83]:
# preview one of the panels
panel_summary['DNA Tubes - no sets']

year,2013,2014,2015,2016,2017,2018
month,,,,,,
1900-01-01,1062.0,654.0,408.0,408.0,682.0,469.0
1900-02-01,805.0,775.0,394.0,686.0,491.0,542.0
1900-03-01,586.0,592.0,670.0,541.0,747.0,0.0
1900-04-01,955.0,790.0,1186.0,679.0,515.0,0.0
1900-05-01,795.0,675.0,702.0,518.0,433.0,0.0
1900-06-01,960.0,459.0,666.0,1120.0,562.0,0.0
1900-07-01,482.0,740.0,489.0,600.0,343.0,0.0
1900-08-01,580.0,650.0,470.0,253.0,456.0,0.0
1900-09-01,869.0,581.0,428.0,1123.0,547.0,0.0


#### To keep from repeating ourselves, we can create a function that takes in a dataframe and outputs the formatted data table

In [86]:
def create_data_table(df):
    # specify the what fields and names to use for each of the columns
    columns = [
        TableColumn(field="month", title="Month"),
        TableColumn(field="2013", title="2013"),
        TableColumn(field="2014", title="2014"),
        TableColumn(field="2015", title="2015"),
        TableColumn(field="2016", title="2016"),
        TableColumn(field="2017", title="2017"),
        TableColumn(field="2018", title="2018")
    ]
    # create an empty ColumnDataSource
    source = ColumnDataSource()
    # transfer the data from the dataframe to the ColumnDataSource
    source.data = {
        'month' : df.reset_index().month.dt.strftime('%b'),
        '2013'  : df[2013],
        '2014' : df[2014],
        '2015' : df[2015],
        '2016' : df[2016],
        '2017' : df[2017],
        '2018' : df[2018]
    }
    # create a new DataTable with the data and Columns we specified
    return DataTable(source=source, columns=columns, width=800)

In [101]:
# create a dictionary to store the combined plot + DataTable layouts
table_layouts = dict()
# loop thru the plots and add the DataTables
for dataset_name in layouts:
    table_layouts[dataset_name] = (Row(layouts[dataset_name], create_data_table(panel_summary[dataset_name])))

#### Combine the individual plots into a single page with tabs for each plot

In [102]:
tabs = Tabs(tabs=[Panel(child=layout, title=layout_name) for layout_name, layout in table_layouts.items()])

In [103]:
# add file extension to the graphs title we decided at the beginning
graphs_filename = graphs_title + ".html"
# save as file
output_file(graphs_filename, title=graphs_title)
# to see the plot, uncomment the line below by removing the '#' from the beginning
show(tabs)

W-1004 (BOTH_CHILD_AND_ROOT): Models should not be a document root if they are in a layout box: DataTable(id='015e9cf8-1d96-460a-b615-7c7a5c6a0458', ...)
W-1004 (BOTH_CHILD_AND_ROOT): Models should not be a document root if they are in a layout box: Figure(id='cf1da12b-74c7-486c-bfb7-8d98a9065def', ...)
W-1004 (BOTH_CHILD_AND_ROOT): Models should not be a document root if they are in a layout box: Figure(id='40133e32-67dd-438f-887c-0f0127c58bf0', ...)
W-1004 (BOTH_CHILD_AND_ROOT): Models should not be a document root if they are in a layout box: Figure(id='cb1964cb-3a12-4b0e-a165-99cc306fd9b6', ...)
W-1004 (BOTH_CHILD_AND_ROOT): Models should not be a document root if they are in a layout box: Figure(id='cf1da12b-74c7-486c-bfb7-8d98a9065def', ...)
W-1004 (BOTH_CHILD_AND_ROOT): Models should not be a document root if they are in a layout box: Figure(id='40133e32-67dd-438f-887c-0f0127c58bf0', ...)
W-1004 (BOTH_CHILD_AND_ROOT): Models should not be a document root if they are in a layout 

### Finally, we can save the summary tables to excel

In [ ]:
# save it to an excel file
panel_summary.to_excel('dna_stats.xlsx')